## Yahoo Fundamental

In [24]:
# def get_yahoo_economic_calendar(table_class: str = None) -> pd.DataFrame:
#     """
#     Extrat data from trading economics callendar.

#     Parameters:
#         classe_tabela (str): Class CSS from table to scrap (optional).

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get("https://tradingeconomics.com/calendar", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')

#         if table_class:
#             tabela = soup.find('table', {'class': table_class})
#         else:
#             tabela = soup.find('table')
        
#         headers = [th.text.strip() for th in tabela.find_all('th')]
#         print(headers)
#         rows = []
#         for row in tabela.find_all('tr')[1:]:
#             cols = [td.text.strip() for td in row.find_all('td')]
#             if cols: 
#                 rows.append(cols)

#         df = pd.DataFrame(rows, columns=headers if headers else None)
        

#     except requests.exceptions.RequestException as e:
#         print(f"Error accessing URL: {e}")
#     except Exception as e:
#         print(f"Error processing data: {e}")

#     return df


In [1]:
# def get_whalewisdom(ticker: str) -> pd.DataFrame:
#     """
#     Extracts data from WhaleWisdom for a given stock ticker.

#     Parameters:
#         ticker (str): Stock ticker symbol.

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get(f"https://whalewisdom.com/stock/{ticker}", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         # Encontrar a div com a classe "v-data-table__wrapper"
#         div_wrapper = soup.find('div', {'class': 'v-window__container'})
       
#         if not div_wrapper:
#             raise ValueError("Div com a classe 'v-window__container' não encontrada.")

#         # Buscar a primeira tabela dentro desta div
#         tabela = div_wrapper.find('table')
#         if not tabela:
#             raise ValueError("Nenhuma tabela encontrada dentro da div especificada.")
        
#         # Coletar os cabeçalhos da tabela ignorando ícones
#         headers = [th.find('span').text.strip() for th in tabela.find_all('th') if th.find('span')]
#         # print(headers)

#         # Encontrando todas as linhas dentro da tabela (tbody > tr)
#         rows = tabela.find("tbody").find_all("tr")
#         print(rows)
#         # Extraindo os dados de cada linha
#         data = []
#         for row in rows:
#             columns = row.find_all("td")

#             # Pegando os valores correspondentes
#             institution_name = columns[0].text.strip()
#             shares_held = columns[2].text.strip()
#             portfolio_value = columns[3].text.strip()
#             percentage_ownership = columns[4].text.strip()
#             last_report_date = columns[-1].text.strip()

#             data.append({
#                 "Instituição": institution_name,
#                 "Ações Detidas": shares_held,
#                 "Valor do Portfólio": portfolio_value,
#                 "Percentual de Participação": percentage_ownership,
#                 "Última Atualização": last_report_date
#             })
#         #print(data)
    
#     except requests.exceptions.RequestException as e:
#         print(f"Erro ao acessar a URL: {e}")
#     except ValueError as e:
#         print(f"Erro na extração de dados: {e}")
#     except Exception as e:
#         print(f"Erro inesperado: {e}")
    
#     return pd.DataFrame()

In [2]:
# df_whale_s = get_whalewisdom(ticker = "ctre")
# df_whale_s

In [3]:
# base_url = 'https://whalewisdom.com/shell/command.html?args=%7B%22command%22:%22filer_lookup%22,%20%22name%22:%22berkshire%22%7D'
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
#     "Accept": "application/json"
# }

# response = requests.get(base_url, headers=headers)
# response.raise_for_status()

# response.text

In [4]:
# import hashlib
# import hmac
# import base64
# import json
# import time
# import urllib.parse
# import requests

# class WhaleWisdom:
#     def __init__(self, secret_key, shared_key):
#         self.secret_key = secret_key
#         self.shared_key = shared_key
#         self.timestamp = self.get_timestamp()

#     def get_timestamp(self):
#         """Gera o timestamp no formato UTC"""
#         return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

#     def signature(self, args):
#         """Gera a assinatura HMAC-SHA1 em Base64"""
#         message = f"{args}\n{self.timestamp}".encode('utf-8')
#         hmac_digest = hmac.new(self.secret_key.encode('utf-8'), message, hashlib.sha1).digest()
#         return base64.b64encode(hmac_digest).decode('utf-8')

#     def encode(self, string):
#         """Codifica a string para URL"""
#         return urllib.parse.quote(string, safe='')

#     def endpoint(self, args):
#         """Constrói a URL da API do WhaleWisdom"""
#         encoded_args = self.encode(json.dumps(args))
#         api_sig = self.signature(json.dumps(args))
#         encoded_timestamp = self.encode(self.timestamp)

#         return f"https://whalewisdom.com/shell/command.json?args={encoded_args}&api_shared_key={self.shared_key}&api_sig={api_sig}&timestamp={encoded_timestamp}"

#     def request(self, args):
#         """Faz a requisição à API"""
#         url = self.endpoint(args)
#         response = requests.get(url)

#         if response.status_code == 200:
#             return response.text
#         else:
#             print(f"Erro {response.status_code}: {response.text}")
#             return None


# # **Exemplo de Uso**
# WW_SHARED_KEY = "aTWxWrcILEOGqkAPZtAL"
# WW_SECRET_KEY = "JGlAleUrGy1bWaLoiWwESNIbTHaovA0ocKOnr4MW"

# ww = WhaleWisdom(secret_key=WW_SECRET_KEY, shared_key=WW_SHARED_KEY)

# # Obter as participações (holdings) de um gestor específico
# response = ww.request({
#     "command": "stock_lookup",
#     "symbol": "ctre"
# })

# data = json.loads(response)

# stock_id = data["stocks"][0]["id"]
# name = data["stocks"][0]["name"]
# status = data["stocks"][0]["status"]
# link = data["stocks"][0]["link"]

# stock_id

In [5]:
# response = ww.request(
# {"command":"holdings","stock_ids":[167941],"filer_ids":[373], "limit": 10, "all_quarters":0})
# response


In [6]:
# response

## STOCKS



In [7]:
import yfinance as yf
import pandas as pd
import talib
import numpy as np
from django.http import JsonResponse
from datetime import datetime, timedelta
from scipy.signal import argrelextrema

# S&P 500 → ^GSPC
# Dow Jones → ^DJI
# Nasdaq 100 → ^NDX
# Russell 2000 → ^RUT
# DAX (Alemanha) → ^GDAXI
# Definir o ticker da ação
symbol = "AAPL"  # Apple Inc.
# symbol = "^GSPC"  # Apple Inc.
# symbol = "XLK"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários

yahoo_symbol_info = yf.Ticker(symbol).info

In [8]:
def get_data_history(symbol : str, period : str, interval : str, start = '1900-01-01', end = datetime.now(), prepost : bool = True):
        '''
        Data collection from yahoo

        Parameters:
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime, inclusive. Default is 1900-01-01 E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
        end: str
            Download end date string (YYYY-MM-DD) or _datetime, exclusive. Default is now E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
        prepost : bool
            Include Pre and Post market data in results? Default is False
        '''
        yahoo_data_history = yf.Ticker(symbol).history(period=period, interval=interval, start=start, end=end, prepost=prepost)
        yahoo_data_history.reset_index(inplace=True)

        yahoo_data_history["Date"] = yahoo_data_history["Date"].dt.strftime("%Y-%m-%d %H:%M")

        return yahoo_data_history

data_history = get_data_history("EURCAD=X", period, interval)

In [9]:
# def get_sector_etf_info(sector_name, search_value: str = "info" ):
#     """
#     Return information about symbol ETF sector.
#     """
#     sector_map = {
#         "Technology": "XLK",
#         "Financial Services": "XLF",
#         "Consumer Cyclical": "XLY",
#         "Healthcare": "XLV",
#         "Communication Services": "XLC",
#         "Industrials": "XLI",
#         "Consumer Defensive": "XLP",
#         "Energy": "XLE",
#         "Real Estate": "XLRE",
#         "Basic Materials": "XLB",
#         "Utilities": "XLU",
#     }

#     etf_symbol = sector_map.get(sector_name)
#     if not etf_symbol:
#         return "N/A"

#     ticker = yf.Ticker(etf_symbol)

#     if search_value == "info":
#         return ticker.info
#     else:
#         return ticker.info.get(search_value, "N/A")


In [10]:
# def get_yoy_metric(series):
#     if isinstance(series, pd.Series):
#         series = series.replace(0, np.nan).dropna()

#         if len(series) >= 2:
#             yoy_growth = ((series - series.shift(-1)) / series.shift(-1)) * 100
#             yoy_growth = yoy_growth.dropna()
#         else:
#             yoy_growth = pd.Series("N/A", index=series.index)

#     return yoy_growth


# def get_cagr_metric(series):
#     if isinstance(series, pd.Series):
#         series = pd.to_numeric(series, errors='coerce').dropna()

#         if len(series) >= 2:
#             start_value = series.iloc[-1]
#             end_value = series.iloc[0]
#             num_years = len(series) - 1

#             if start_value != 0:
#                 cagr = ((end_value / start_value) ** (1 / num_years)) - 1
#             else:
#                 cagr = "N/A"
#         else:
#             cagr = "N/A"
#     else:
#         cagr = "N/A"

#     cagr_percent = cagr * 100 if cagr != "N/A" else "N/A"

#     return cagr_percent

In [11]:
# yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
# yahoo_symbol_cashflow.index

In [12]:
# yahoo_symbol_income = yf.Ticker(symbol).income_stmt
# yahoo_symbol_income.index

In [13]:
# yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
# yahoo_symbol_balancesheet.index

In [14]:
# def get_symbol_fundamental_info(symbol : str):
#         '''
#         Return detailed fundamental information about asset
#         '''
#         try:
#             yahoo_symbol_info = yf.Ticker(symbol).info
#         except:
#             yahoo_symbol_info = {}
#         try:
#             yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
#         except:
#             yahoo_symbol_balancesheet = pd.DataFrame()
#         try:
#             yahoo_symbol_income = yf.Ticker(symbol).income_stmt
#         except:
#             yahoo_symbol_income = pd.DataFrame()
#         try:
#             yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
#         except:
#             yahoo_symbol_cashflow = pd.DataFrame()
        
#         # Valuation
#         sector = yahoo_symbol_info.get("sector")

#         sector_pe = get_sector_etf_info(sector, "trailingPE")

#         # Dividends & BuyBacks
#         eps_ann = yahoo_symbol_info.get("epsCurrentYear", "N/A")
#         dividend_rate = yahoo_symbol_info.get("dividendRate", "N/A")
#         if eps_ann != "N/A" and dividend_rate != "N/A" and dividend_rate != 0:
#             div_coverage_rate = eps_ann / dividend_rate
#         else:
#             div_coverage_rate = "N/A" 

#         # Profitability
#         # net_income
#         if 'Net Income' in yahoo_symbol_income.index:
#             net_income = yahoo_symbol_income.loc['Net Income']
#             if pd.isna(net_income).all():
#                 net_income = "N/A"

#         # total_revenue
#         if 'Total Revenue' in yahoo_symbol_income.index:
#             total_revenue = yahoo_symbol_income.loc['Total Revenue']
#             if pd.isna(total_revenue).all():
#                 total_revenue = "N/A"

#         # cost_of_revenue
#         if 'Cost Of Revenue' in yahoo_symbol_income.index:
#             cost_of_revenue = yahoo_symbol_income.loc['Cost Of Revenue']
#             if pd.isna(cost_of_revenue).all():
#                 cost_of_revenue = "N/A"

#         # gross_profit
#         if 'Gross Profit' in yahoo_symbol_income.index:
#             gross_profit = yahoo_symbol_income.loc['Gross Profit']
#             if pd.isna(gross_profit).all():
#                 gross_profit = "N/A"

#         # operating_expenses
#         if 'Operating Expense' in yahoo_symbol_income.index:
#             operating_expenses = yahoo_symbol_income.loc['Operating Expense']
#             if pd.isna(operating_expenses).all():
#                 operating_expenses = "N/A"
                            
#         yoy_cost_of_revenue = get_yoy_metric(cost_of_revenue)
#         yoy_total_revenue = get_yoy_metric(total_revenue)
#         yoy_operating_expenses = get_yoy_metric(operating_expenses)

#         cagr_cost_of_revenue = get_cagr_metric(cost_of_revenue)
#         cagr_total_revenue = get_cagr_metric(total_revenue)
#         cagr_operating_expenses = get_cagr_metric(operating_expenses)

#         # Growth & NetWorth & Health
#         # total_assets
#         if 'Total Assets' in yahoo_symbol_balancesheet.index:
#             total_assets = yahoo_symbol_balancesheet.loc['Total Assets']
#             if pd.isna(total_assets).all():
#                 total_assets = "N/A"
        
#         # current_liabilities
#         if 'Current Liabilities' in yahoo_symbol_balancesheet.index:
#             current_liabilities = yahoo_symbol_balancesheet.loc['Current Liabilities']
#             if pd.isna(current_liabilities).all():
#                 current_liabilities = "N/A"

#         # non_current_liabilities
#         if 'Total Non Current Liabilities Net Minority Interest' in yahoo_symbol_balancesheet.index:
#             non_current_liabilities = yahoo_symbol_balancesheet.loc['Total Non Current Liabilities Net Minority Interest']
#             if pd.isna(non_current_liabilities).all():
#                 non_current_liabilities = "N/A"

#         # total_liabilities
#         if not current_liabilities.isna().all() and not non_current_liabilities.isna().all():
#             total_liabilities = (current_liabilities + non_current_liabilities)
#         else:
#             total_liabilities = "N/A"

#         # net_worth
#         if not total_assets.isna().all() and not current_liabilities.isna().all() and not non_current_liabilities.isna().all():
#             total_assets = total_assets.replace(0, np.nan)
#             net_worth = total_assets.iloc[0] - total_liabilities.iloc[0]
#         else:
#             net_worth = "N/A"

#         # current_assets
#         if 'Current Assets' in yahoo_symbol_balancesheet.index:
#             current_assets = yahoo_symbol_balancesheet.loc['Current Assets']
#             if pd.isna(current_assets).all():
#                 current_assets = "N/A"

#         # non_current_assets
#         if 'Total Non Current Assets' in yahoo_symbol_balancesheet.index:
#             non_current_assets = yahoo_symbol_balancesheet.loc['Total Non Current Assets']
#             if pd.isna(non_current_assets).all():
#                 non_current_assets = "N/A"
        
#         # short_term_debt_coverage
#         if not current_assets.isna().all() and not current_liabilities.isna().all():
#             short_term_debt_coverage = current_assets - current_liabilities
#         else:
#             short_term_debt_coverage = "N/A"

#         # long_term_debt_coverage
#         if not non_current_assets.isna().all() and not non_current_liabilities.isna().all():
#             long_term_debt_coverage = non_current_assets - non_current_liabilities
#         else:
#             long_term_debt_coverage = "N/A"

#         cagr_total_liabilities = get_cagr_metric(total_liabilities)
#         cagr_total_assets = get_cagr_metric(total_assets)

#         # cash_cash_equivalents
#         if 'Cash And Cash Equivalents' in yahoo_symbol_balancesheet.index:
#             cash_cash_equivalents = yahoo_symbol_balancesheet.loc['Cash And Cash Equivalents']
#             if pd.isna(cash_cash_equivalents).all():
#                 cash_cash_equivalents = "N/A"

#         # stockholders_equity
#         if 'Stockholders Equity' in yahoo_symbol_balancesheet.index:
#             stockholders_equity = yahoo_symbol_balancesheet.loc['Stockholders Equity']
#             if pd.isna(stockholders_equity).all():
#                 stockholders_equity = "N/A"
        
#         cagr_stockholder_equity = get_cagr_metric(stockholders_equity)
        
#         # Cashflow
#         # free_cashflow
#         if 'Free Cash Flow' in yahoo_symbol_cashflow.index:
#             free_cashflow = yahoo_symbol_cashflow.loc['Free Cash Flow']
#             if pd.isna(free_cashflow).all():
#                 free_cashflow = "N/A"
        
#         # operating_cashflow
#         if 'Operating Cash Flow' in yahoo_symbol_cashflow.index:
#             operating_cashflow = yahoo_symbol_cashflow.loc['Operating Cash Flow']
#             if pd.isna(operating_cashflow).all():
#                 operating_cashflow = "N/A"

#         # capital_expenditure
#         if 'Capital Expenditure' in yahoo_symbol_cashflow.index:
#             capital_expenditure = yahoo_symbol_cashflow.loc['Capital Expenditure']
#             if pd.isna(capital_expenditure).all():
#                 capital_expenditure = "N/A"

#         # market_cap
#         market_cap = yahoo_symbol_info.get('marketCap', "N/A")

#         # free_cashflow_yield
#         if not pd.isna(market_cap) and market_cap != 0 and not free_cashflow.isna().all():
#             free_cashflow_yield = ((free_cashflow.iloc[0] / market_cap) * 100)
#         else:
#             free_cashflow_yield = "N/A"

#         # Ratios
#         # current_ratio
#         if not current_assets.isna().all() and not current_liabilities.isna().all():
#             current_liabilities = current_liabilities.replace(0, np.nan)
#             current_ratio = ((current_assets / current_liabilities) * 100).fillna("N/A")
#         else:
#             current_ratio = "N/A"
        
#         # cagr_current_ratio
#         cagr_current_ratio = get_cagr_metric(current_ratio)
        
#         # cash_ratio
#         if not cash_cash_equivalents.isna().all() and not current_liabilities.isna().all():
#             current_liabilities = current_liabilities.replace(0, np.nan)
#             cash_ratio = ((cash_cash_equivalents / current_liabilities) * 100).fillna("N/A")
#         else:
#             cash_ratio = "N/A"
        
#         # cagr_cash_ratio
#         cagr_cash_ratio = get_cagr_metric(cash_ratio)

#         # gross_margin
#         if not gross_profit.isna().all() and not total_revenue.isna().all():
#             total_revenue = total_revenue.replace(0, np.nan)
#             gross_margin = ((gross_profit / total_revenue) * 100).fillna("N/A")
#         else:
#             gross_margin = "N/A"

#         # cagr_gross_margin
#         # cagr_gross_margin = fm.get_cagr_metric(gross_margin)

#         # operating_income
#         if 'Operating Income' in yahoo_symbol_income.index:
#             operating_income = yahoo_symbol_income.loc['Operating Income']
#             if pd.isna(operating_income).all():
#                 operating_income = "N/A"

#         # operation_margin
#         if not operating_income.isna().all() and not total_revenue.isna().all():
#             total_revenue = total_revenue.replace(0, np.nan)
#             operating_margin = ((operating_income / total_revenue) * 100).fillna("N/A")
#         else:
#             operating_margin = "N/A"

#         # cagr_operating_margin
#         cagr_operating_margin = get_cagr_metric(operating_margin)

#         # profit_margin
#         if not net_income.isna().all() and not total_revenue.isna().all():
#             total_revenue = total_revenue.replace(0, np.nan)
#             profit_margin = ((net_income / total_revenue) * 100).fillna("N/A")
#         else:
#             profit_margin = "N/A"

#         # cagr_profit_margin
#         cagr_profit_margin = get_cagr_metric(profit_margin)
        
#         # return_on_equity
#         if not net_income.isna().all() and not stockholders_equity.isna().all():
#             stockholders_equity = stockholders_equity.replace(0, np.nan)
#             return_on_equity = ((net_income / stockholders_equity) * 100).fillna("N/A")
#         else:
#             return_on_equity = "N/A"

#         # cagr_return_on_equity
#         cagr_return_on_equity = get_cagr_metric(return_on_equity)
        
#         yahoo_symbol_fundamental_info = {
#             "valuation": {
#                 "trailingPE": yahoo_symbol_info.get("trailingPE", "N/A"),
#                 "sectorTrailingPE": sector_pe,
#                 "forwardPE": yahoo_symbol_info.get("forwardPE", "N/A"),
#                 "PEGRatio": yahoo_symbol_info.get("trailingPegRatio", "N/A"),
#             },
#             "dividends": {
#                 "divCoverageRate": div_coverage_rate,
#                 "dividendYield": yahoo_symbol_info.get("dividendYield", "N/A"),
#                 "fiveYearAvgDividendYield": yahoo_symbol_info.get("fiveYearAvgDividendYield", "N/A"),
#             },
#             "profitability": {
#                 "NetIncome": net_income.iloc[0],
#                 "TotalRevenue": total_revenue.iloc[0],
#                 "CostOfRevenue": cost_of_revenue.iloc[0],
#                 "GrossProfit": gross_profit.iloc[0],
#                 "OperatingExpenses": operating_expenses.iloc[0],
#                 "CostOfRevenueCAGR": cagr_cost_of_revenue,
#                 "TotalRevenueCAGR": cagr_total_revenue,
#                 "OperatingExpensesCAGR": cagr_operating_expenses,
#                 # "CostOfRevenueYOY": yoy_cost_of_revenue,
#                 # "TotalRevenueYOY": yoy_total_revenue,
#             },
#             "liquidity": {
#                 # Actual
#                 "TotalAssets": total_assets.iloc[0],
#                 "TotalLiabilities": total_liabilities.iloc[0],
#                 "NetWorth": net_worth,
#                 "CashCashEquivalents": cash_cash_equivalents.iloc[0],
#                 # Short Term 1y
#                 "ShortTermDebtCoverage" : short_term_debt_coverage.iloc[0],
#                 "CurrentAssets": current_assets.iloc[0],
#                 "CurrentLiabilities": current_liabilities.iloc[0],
#                 # Long Term
#                 "LongTermDebtCoverage" : long_term_debt_coverage.iloc[0],
#                 "NonCurrentAssets": non_current_assets.iloc[0],
#                 "NonCurrentLiabilities": non_current_liabilities.iloc[0],
#                 # Growth
#                 "TotalAssetsCAGR": cagr_total_assets,
#                 "TotalLiabilitiesCAGR": cagr_total_liabilities,
#                 # Stockholders Equity
#                 "StockholdersEquityCAGR": cagr_stockholder_equity,
#                 "StockholdersEquity": stockholders_equity.iloc[0],
#             },
#             "cashflow": {
#                 "FreeCashflow": free_cashflow.iloc[0],
#                 "OperatingCashflow": operating_cashflow.iloc[0],
#                 "CapitalExpenditure": capital_expenditure.iloc[0],
#                 "MarketCap": market_cap,
#                 "FreeCashflowYield": free_cashflow_yield,
#             },
#             "ratios": {
#                 # Health & Debt
#                 "CurrentRatio": current_ratio.iloc[0],
#                 "CurrentRatioCAGR": cagr_current_ratio,
#                 "CashRatio": cash_ratio.iloc[0],
#                 "CashRatioCAGR": cagr_cash_ratio,
#                 # Margins
#                 "GrossMargin": gross_margin.iloc[0],
#                 # "GrossMarginCAGR": cagr_gross_margin,
#                 "OperatingMargin": operating_margin.iloc[0],
#                 "OperatingMarginCAGR": cagr_operating_margin,
#                 "ProfitMargin": profit_margin.iloc[0],
#                 "ProfitMarginCAGR": cagr_profit_margin,
#                 "ReturnOnEquity": return_on_equity.iloc[0],
#                 "ReturnOnEquityCAGR": cagr_return_on_equity,
#             },
#             "market_risk_and_sentiment": {
#                 "beta": yahoo_symbol_info.get("beta", "N/A"),
#                 "auditRisk": yahoo_symbol_info.get("auditRisk", "N/A"),
#                 "boardRisk": yahoo_symbol_info.get("boardRisk", "N/A"),
#                 "sharesPercentSharesOut": yahoo_symbol_info.get("sharesPercentSharesOut", "N/A"),
#                 "recommendationMean": yahoo_symbol_info.get("recommendationMean", "N/A"),
#                 "targetMeanPrice": yahoo_symbol_info.get("targetMeanPrice", "N/A")
#             }
#         }
#         return yahoo_symbol_fundamental_info


In [15]:
def peak_detect(close, order=5):
    '''
    This function find the peaks of a timeframe    
    '''

    if len(close) < 5:
        return [], [], 0, 0, []

    if isinstance(close, pd.Series):
        close_np = close.values  # Converte apenas os valores se for uma série Pandas
    else:
        close_np = np.array(close)

    max_idx = list(argrelextrema(close_np, comparator=np.greater, order=order)[0])
    min_idx = list(argrelextrema(close_np, comparator=np.less, order=order)[0])

    idx = max_idx + min_idx + [len(close_np)-1]
    idx.sort()

    current_idx = idx[-5:]

    start = min(current_idx)
    end = max(current_idx)

    current_pat = close_np[current_idx]

    return current_idx, current_pat, start, end, idx

In [16]:
current_idx, current_pat, start, end, idx = peak_detect(data_history.Close)

In [17]:
import plotly.graph_objects as go

# Criar gráfico com Plotly
fig = go.Figure()

# Linha principal (Série Temporal)
fig.add_trace(go.Scatter(
    x=data_history['Date'], 
    y=data_history['Close'], 
    mode='lines', 
    name='Preço de Fechamento'
))

# Destacar Picos e Vales
fig.add_trace(go.Scatter(
    x=data_history['Date'].iloc[current_idx], 
    y=current_pat, 
    mode='markers', 
    marker=dict(size=10, color='red'), 
    name='Picos/Vales'
))

# Configuração do layout
fig.update_layout(
    title='Detecção de Picos e Vales - Série Temporal',
    xaxis_title='Data',
    yaxis_title='Preço de Fechamento',
    template='plotly_dark',  # Escolha visual moderna e clara
    height=600,
    width=1000
)

# Exibir gráfico
fig.show()

In [18]:
def get_gartley_hp(moves, err_allowed):
        
    XA = moves[0]
    AB = moves[1]
    BC = moves[2]
    CD = moves[3]
                
    AB_range = np.array([.618 - err_allowed, .618 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([1.27 - err_allowed, 1.618 + err_allowed]) * abs(BC)


    # peak information
    peaks = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None
    }

    ## === Bulish Butterfly === ## 
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:

        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:
            peaks["XA"] = XA
            peaks["AB"] = AB
            peaks["BC"] = BC
            peaks["CD"] = CD
            return 1, peaks  # Bullish Crab
        else:
            return np.NAN, peaks

    ## === Bearish Butterfly === ## 
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:

        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:
            peaks["XA"] = XA
            peaks["AB"] = AB
            peaks["BC"] = BC
            peaks["CD"] = CD
            return -1, peaks  # Bullish Crab
        else:
            return np.NAN

    else:
        return np.NAN 

In [19]:
def get_butterfly_hp(moves, err_allowed):
        
    XA = moves[0]
    AB = moves[1]
    BC = moves[2]
    CD = moves[3]
                
    AB_range = np.array([.786 - err_allowed, .786 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([1.618 - err_allowed, 2.618 + err_allowed]) * abs(BC)


    # peak information
    peaks = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None
    }

    ## === Bulish Butterfly === ## 
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:

        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:
            peaks["XA"] = XA
            peaks["AB"] = AB
            peaks["BC"] = BC
            peaks["CD"] = CD
            return 1, peaks  # Bullish Crab
        else:
            return np.NAN, peaks

    ## === Bearish Butterfly === ## 
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:

        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:
            peaks["XA"] = XA
            peaks["AB"] = AB
            peaks["BC"] = BC
            peaks["CD"] = CD
            return -1, peaks  # Bullish Crab
        else:
            return np.NAN

    else:
        return np.NAN 

In [20]:
def get_bat_hp(moves, err_allowed):
        
    XA = moves[0]
    AB = moves[1]
    BC = moves[2]
    CD = moves[3]
                
    AB_range = np.array([.382 - err_allowed, .5 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([1.618 - err_allowed, 2.618 + err_allowed]) * abs(BC)


    # peak information
    peaks = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None
    }

    ## === Bulish Butterfly === ## 
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:

        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:
            peaks["XA"] = XA
            peaks["AB"] = AB
            peaks["BC"] = BC
            peaks["CD"] = CD
            return 1, peaks  # Bullish Crab
        else:
            return np.NAN, peaks

    ## === Bearish Butterfly === ## 
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:

        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:
            peaks["XA"] = XA
            peaks["AB"] = AB
            peaks["BC"] = BC
            peaks["CD"] = CD
            return -1, peaks  # Bullish Crab
        else:
            return np.NAN

    else:
        return np.NAN 

In [21]:
def get_crab_hp(moves, err_allowed):
        
    XA = moves[0]
    AB = moves[1]
    BC = moves[2]
    CD = moves[3]
                
    AB_range = np.array([.382 - err_allowed, .618 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([2.24 - err_allowed, 3.618 + err_allowed]) * abs(BC)

    # peak information
    targets = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None
    }

    ## === Bulish Butterfly === ## 
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:

        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:
            targets["XA"] = XA # 3rd Target
            targets["AB"] = AB # 1st Target
            targets["BC"] = BC # 2nd Target
            targets["CD"] = CD # Last Break Point
            
            return 1 # Buy
        else:
            return np.NAN

    ## === Bearish Butterfly === ## 
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:

        if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:
            targets["XA"] = XA # 3rd Target
            targets["AB"] = AB # 1st Target
            targets["BC"] = BC # 2nd Target
            targets["CD"] = CD # Last Break Ppint
            
            return -1 # Sell
        else:
            return np.NAN

    else:
        return np.NAN 

In [52]:
import plotly.graph_objects as go
from tqdm import tqdm

def get_harmonic_pattern(data, err_allowed=0.01, order=5):
    """
    Função para desenhar os padrões Gartley Harmonic usando Plotly
    """

    close = data.Close

    err_allowed = err_allowed
    last_detected_pattern = None

    # Criar figura inicial
    fig = go.Figure()

    # Gráfico principal (preço)
    fig.add_trace(go.Scatter(
        x=data.index,
        y=close,
        mode='lines',
        name='Preço',
        line=dict(color='blue')
    ))

    for i in tqdm(range(100, len(close))):
        current_idx, current_pat, start, end, idx = peak_detect(close.values[:i], order=order)
        
        XA = current_pat[1] - current_pat[0]
        AB = current_pat[2] - current_pat[1]
        BC = current_pat[3] - current_pat[2]
        CD = current_pat[4] - current_pat[3]

        moves = [XA, AB, BC, CD]

        gart = get_gartley_hp(moves, err_allowed)
        butt = get_butterfly_hp(moves, err_allowed)
        bat = get_bat_hp(moves, err_allowed)
        crab = get_crab_hp(moves, err_allowed)

        harmonics = np.array([gart, butt, bat, crab])
        labels = ['Gartley', 'Butterfly', 'Bat', 'Crab']


        if np.any(harmonics == 1) or np.any(harmonics == -1):
            for j in range(0, len(harmonics)):
                if harmonics[j] == 1 or harmonics[j] == -1:
                    signal = 'Bearish ' if harmonics[j] == -1 else 'Bullish '
                    label = signal + labels[j] + ' Found'

                    start = min(current_idx)
                    end = max(current_idx)
                    date = data.iloc[end].name

                    last_detected_pattern = {
                        "start": start,
                        "end": i,
                        "current_idx": current_idx,
                        "current_pat": current_pat,
                        "label": label
                    }

                    # Desenhar o padrão com pontos
                    fig.add_trace(go.Scatter(
                        x=data.index[current_idx],
                        y=current_pat,
                        mode='markers+lines',
                        name=label,
                        marker=dict(size=10, color='red' if harmonics[j] == -1 else 'green')
                    ))

    if last_detected_pattern:
        print("✅ Padrão Harmônico Detetado:", last_detected_pattern['label'])
    else:
        print("❌ Nenhum padrão detetado.")

    fig.update_layout(
        title="Detecção de Padrão Harmônico",
        xaxis_title="Data",
        yaxis_title="Preço de Fecho",
        legend_title="Padrões Detectados",
        template="plotly_dark"
    )


    fig.show()


In [53]:
get_harmonic_pattern(data_history, err_allowed=0.02, order=5)

  0%|          | 0/5434 [00:00<?, ?it/s]C:\Users\joaom\AppData\Local\Temp\ipykernel_14708\3853674431.py:41: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

100%|██████████| 5434/5434 [00:02<00:00, 2030.02it/s]

✅ Padrão Harmônico Detetado: Bearish Crab Found


In [11]:
# https://medium.com/towards-data-science/pivot-points-calculation-in-python-for-day-trading-659c1e92d323


# ### ZONAS PIVOT! ###

## Data Collection Binance


In [ ]:
# import requests

# url = "https://api.binance.com/api/v3/ticker/price"
# params = {"symbol": "BTCUSDT"}

# response = requests.get(url, params=params)

# if response.status_code == 200:
#     symbol = response.json()["symbol"]
#     price = response.json()["price"]
# else:
#     error = response.status_code

In [ ]:
# from decimal import Decimal
# from datetime import datetime


# def get_crypto_symbol_24h(symbol : str):

#     url = "https://api.binance.com/api/v3/ticker/24hr"
#     params = {"symbol": symbol}

#     response = requests.get(url, params=params)

#     if response.status_code == 200:
#         priceChangePercent = Decimal(response.json()["priceChangePercent"])
#         weightedAvgPrice = Decimal(response.json()["weightedAvgPrice"])
#         prevClosePrice = Decimal(response.json()["prevClosePrice"])
#         priceChange = Decimal(response.json()["priceChange"])
#         lastPrice = Decimal(response.json()["lastPrice"])
#         lastQty = Decimal(response.json()["lastQty"])
#         bidPrice = Decimal(response.json()["bidPrice"])
#         bidQty = Decimal(response.json()["bidQty"])
#         askPrice = Decimal(response.json()["askPrice"])
#         askQty = Decimal(response.json()["askQty"])
#         openPrice = Decimal(response.json()["openPrice"])
#         highPrice = Decimal(response.json()["highPrice"])
#         lowPrice = Decimal(response.json()["lowPrice"])
#         volume = Decimal(response.json()["volume"])
#         quoteVolume = Decimal(response.json()["quoteVolume"])
#         openTime = datetime.fromtimestamp(response.json()["openTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
#         closeTime = datetime.fromtimestamp(response.json()["closeTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
#         firstId = response.json()["firstId"]
#         lastId = response.json()["lastId"]
#         count = response.json()["count"]
#     else:
#         print(f"Erro: {response.status_code} - {response.text}")

In [ ]:
# url = "https://api.binance.com/api/v3/openOrders"
# params = {"symbol": "BTCUSDT"}

# response = requests.get(url, params=params)
# response.json()
